# Dependency Graph Deep Dive

## 🎯 Mission
Build a complete `DependencyGraph` class from scratch by reading and understanding the actual Odibi implementation.

**Source:** `Odibi/odibi/graph.py`

## Part 1: Understanding the Problem

### The Challenge
You have pipeline nodes with dependencies:
```python
nodes = [
    NodeConfig(name='extract', depends_on=[]),
    NodeConfig(name='clean', depends_on=['extract']),
    NodeConfig(name='validate', depends_on=['extract']),
    NodeConfig(name='aggregate', depends_on=['clean']),
    NodeConfig(name='report', depends_on=['aggregate'])
]
```

### Questions to Answer
1. What order should nodes execute?
2. Which nodes can run in parallel?
3. Are there any circular dependencies?
4. What happens if a dependency doesn't exist?

In [ ]:
# Setup - Simplified NodeConfig for this lesson
from typing import List, Set, Dict, Optional
from collections import defaultdict, deque
from dataclasses import dataclass

@dataclass
class NodeConfig:
    name: str
    depends_on: List[str]

class DependencyError(Exception):
    """Custom exception for dependency issues."""
    def __init__(self, message: str, cycle: Optional[List[str]] = None):
        super().__init__(message)
        self.cycle = cycle

## Part 2: Graph Construction

### First Principles: Adjacency Lists

Two representations:
1. **Forward (adjacency_list)**: `node → [dependents]`
   - "Who needs me?"
   - Used for: topological sort, execution layers

2. **Reverse (reverse_adjacency_list)**: `node → [dependencies]`
   - "Who do I need?"
   - Used for: finding all dependencies

### The Code

In [ ]:
class DependencyGraph:
    """Builds and analyzes dependency graph from node configurations."""
    
    def __init__(self, nodes: List[NodeConfig]):
        """Initialize dependency graph.
        
        Args:
            nodes: List of node configurations
        """
        # Store nodes by name for O(1) lookup
        self.nodes = {node.name: node for node in nodes}
        
        # Forward edges: dependency → dependents
        self.adjacency_list: Dict[str, List[str]] = defaultdict(list)
        
        # Reverse edges: dependent → dependencies
        self.reverse_adjacency_list: Dict[str, List[str]] = defaultdict(list)
        
        self._build_graph()
        self._validate_graph()
    
    def _build_graph(self) -> None:
        """Build adjacency lists from node dependencies."""
        for node in self.nodes.values():
            for dependency in node.depends_on:
                # Edge from dependency to node (dependency → node)
                self.adjacency_list[dependency].append(node.name)
                self.reverse_adjacency_list[node.name].append(dependency)

### 🧪 Test Graph Construction

In [ ]:
# Simple test pipeline
nodes = [
    NodeConfig('A', []),
    NodeConfig('B', ['A']),
    NodeConfig('C', ['A']),
    NodeConfig('D', ['B', 'C'])
]

# Temporarily skip validation to test construction
class PartialGraph(DependencyGraph):
    def _validate_graph(self): pass

graph = PartialGraph(nodes)

print("Forward (adjacency_list):")
for node, dependents in graph.adjacency_list.items():
    print(f"  {node} → {dependents}")

print("\nReverse (reverse_adjacency_list):")
for node, deps in graph.reverse_adjacency_list.items():
    print(f"  {node} ← {deps}")

## Part 3: Validation

### Two Critical Checks

1. **Missing Dependencies**: All `depends_on` must reference existing nodes
2. **Cycles**: No circular dependencies (A → B → C → A)

In [ ]:
def _validate_graph(self) -> None:
    """Validate the dependency graph.
    
    Raises:
        DependencyError: If validation fails
    """
    self._check_missing_dependencies()
    self._check_cycles()

def _check_missing_dependencies(self) -> None:
    """Check that all dependencies exist as nodes.
    
    Raises:
        DependencyError: If any dependency doesn't exist
    """
    missing_deps = []
    
    for node in self.nodes.values():
        for dependency in node.depends_on:
            if dependency not in self.nodes:
                missing_deps.append((node.name, dependency))
    
    if missing_deps:
        errors = [
            f"Node '{node}' depends on '{dep}' which doesn't exist"
            for node, dep in missing_deps
        ]
        raise DependencyError("Missing dependencies found:\n  " + "\n  ".join(errors))

# Add to DependencyGraph class
DependencyGraph._validate_graph = _validate_graph
DependencyGraph._check_missing_dependencies = _check_missing_dependencies

### Cycle Detection with DFS

**Algorithm:**
1. Track visited nodes (grey = in current path, black = fully processed)
2. For each node, DFS through dependents
3. If we encounter a grey node, cycle found
4. Return the cycle path for helpful error message

In [ ]:
def _check_cycles(self) -> None:
    """Check for circular dependencies.
    
    Raises:
        DependencyError: If cycle detected
    """
    visited = set()  # Fully processed (black)
    rec_stack = set()  # Currently processing (grey)
    
    def visit(node: str, path: List[str]) -> Optional[List[str]]:
        """DFS to detect cycles.
        
        Returns:
            Cycle path if found, None otherwise
        """
        if node in rec_stack:
            # Found a cycle - extract it from path
            cycle_start = path.index(node)
            return path[cycle_start:] + [node]
        
        if node in visited:
            return None  # Already fully processed
        
        visited.add(node)
        rec_stack.add(node)
        path.append(node)
        
        # Visit all dependents
        for dependent in self.adjacency_list[node]:
            cycle = visit(dependent, path[:])
            if cycle:
                return cycle
        
        rec_stack.remove(node)
        return None
    
    # Check from each node (handles disconnected components)
    for node_name in self.nodes.keys():
        if node_name not in visited:
            cycle = visit(node_name, [])
            if cycle:
                raise DependencyError(
                    "Circular dependency detected",
                    cycle=cycle
                )

# Add to DependencyGraph class
DependencyGraph._check_cycles = _check_cycles

### 🧪 Test Validation

In [ ]:
# Test 1: Missing dependency
try:
    bad_nodes = [NodeConfig('A', ['MISSING'])]
    graph = DependencyGraph(bad_nodes)
except DependencyError as e:
    print("✓ Caught missing dependency:")
    print(f"  {e}")

# Test 2: Cycle detection
try:
    cycle_nodes = [
        NodeConfig('A', ['C']),
        NodeConfig('B', ['A']),
        NodeConfig('C', ['B'])
    ]
    graph = DependencyGraph(cycle_nodes)
except DependencyError as e:
    print("\n✓ Caught cycle:")
    print(f"  {e}")
    print(f"  Cycle: {' → '.join(e.cycle)}")

## Part 4: Topological Sort

### Kahn's Algorithm

**Goal:** Find a valid execution order where all dependencies run before dependents.

**Algorithm:**
1. Calculate in-degree (number of dependencies) for each node
2. Add all nodes with in-degree 0 to queue
3. Process queue:
   - Remove node from queue
   - Add to result
   - Decrease in-degree for all dependents
   - Add dependents with in-degree 0 to queue
4. If not all nodes processed, cycle exists

In [ ]:
def topological_sort(self) -> List[str]:
    """Return nodes in topological order (dependencies first).
    
    Uses Kahn's algorithm.
    
    Returns:
        List of node names in execution order
    """
    # Calculate in-degree for each node
    in_degree = {name: 0 for name in self.nodes.keys()}
    for node in self.nodes.values():
        for dependency in node.depends_on:
            in_degree[node.name] += 1
    
    # Queue of nodes with no dependencies
    queue = deque([name for name, degree in in_degree.items() if degree == 0])
    sorted_nodes = []
    
    while queue:
        # Process node with no remaining dependencies
        node_name = queue.popleft()
        sorted_nodes.append(node_name)
        
        # Reduce in-degree for all dependents
        for dependent in self.adjacency_list[node_name]:
            in_degree[dependent] -= 1
            if in_degree[dependent] == 0:
                queue.append(dependent)
    
    # If we didn't process all nodes, there's a cycle
    if len(sorted_nodes) != len(self.nodes):
        raise DependencyError("Failed to create topological sort (likely cycle)")
    
    return sorted_nodes

# Add to DependencyGraph class
DependencyGraph.topological_sort = topological_sort

### 🧪 Test Topological Sort

In [ ]:
nodes = [
    NodeConfig('extract', []),
    NodeConfig('clean', ['extract']),
    NodeConfig('validate', ['extract']),
    NodeConfig('aggregate', ['clean']),
    NodeConfig('report', ['aggregate', 'validate'])
]

graph = DependencyGraph(nodes)
order = graph.topological_sort()

print("Execution order:")
for i, node in enumerate(order, 1):
    deps = graph.nodes[node].depends_on
    deps_str = f" (after: {', '.join(deps)})" if deps else " (no dependencies)"
    print(f"{i}. {node}{deps_str}")

## Part 5: Execution Layers (Parallel Execution)

### The Insight

Topological sort gives us *an* order, but not necessarily the *optimal* parallel execution plan.

**Execution layers** group nodes that can run simultaneously:
- All nodes in a layer have no dependencies on each other
- All nodes in layer N depend only on layers 1..N-1

### Algorithm
Similar to topological sort, but collect all ready nodes at once instead of processing one at a time.

In [ ]:
def get_execution_layers(self) -> List[List[str]]:
    """Group nodes into execution layers for parallel execution.
    
    Nodes in the same layer have no dependencies on each other
    and can run in parallel.
    
    Returns:
        List of layers, where each layer is a list of node names
    """
    # Calculate in-degree for each node
    in_degree = {name: len(node.depends_on) for name, node in self.nodes.items()}
    
    layers = []
    remaining = set(self.nodes.keys())
    
    while remaining:
        # Find all nodes with no remaining dependencies
        current_layer = [name for name in remaining if in_degree[name] == 0]
        
        if not current_layer:
            raise DependencyError("Cannot create execution layers (likely cycle)")
        
        layers.append(current_layer)
        
        # Remove current layer from remaining
        for node_name in current_layer:
            remaining.remove(node_name)
            
            # Reduce in-degree for dependents
            for dependent in self.adjacency_list[node_name]:
                if dependent in remaining:
                    in_degree[dependent] -= 1
    
    return layers

# Add to DependencyGraph class
DependencyGraph.get_execution_layers = get_execution_layers

### 🧪 Test Execution Layers

In [ ]:
layers = graph.get_execution_layers()

print("Execution layers (nodes in same layer run in parallel):\n")
for i, layer in enumerate(layers, 1):
    print(f"Layer {i}: {layer}")
    if len(layer) > 1:
        print(f"  → Can run {len(layer)} nodes in parallel!")

## Part 6: Dependency Queries

### get_dependencies: "What does this node need?"

BFS traversal backward through reverse adjacency list.

In [ ]:
def get_dependencies(self, node_name: str) -> Set[str]:
    """Get all dependencies (direct and transitive) for a node.
    
    Args:
        node_name: Name of node
    
    Returns:
        Set of all dependency node names
    """
    if node_name not in self.nodes:
        raise ValueError(f"Node '{node_name}' not found")
    
    dependencies = set()
    queue = deque([node_name])
    
    while queue:
        current = queue.popleft()
        for dependency in self.reverse_adjacency_list[current]:
            if dependency not in dependencies:
                dependencies.add(dependency)
                queue.append(dependency)
    
    return dependencies

# Add to DependencyGraph class
DependencyGraph.get_dependencies = get_dependencies

### get_dependents: "What needs this node?"

BFS traversal forward through adjacency list.

In [ ]:
def get_dependents(self, node_name: str) -> Set[str]:
    """Get all dependents (direct and transitive) for a node.
    
    Args:
        node_name: Name of node
    
    Returns:
        Set of all dependent node names
    """
    if node_name not in self.nodes:
        raise ValueError(f"Node '{node_name}' not found")
    
    dependents = set()
    queue = deque([node_name])
    
    while queue:
        current = queue.popleft()
        for dependent in self.adjacency_list[current]:
            if dependent not in dependents:
                dependents.add(dependent)
                queue.append(dependent)
    
    return dependents

# Add to DependencyGraph class
DependencyGraph.get_dependents = get_dependents

### get_independent_nodes: Entry points

In [ ]:
def get_independent_nodes(self) -> List[str]:
    """Get nodes that have no dependencies.
    
    Returns:
        List of node names with no dependencies
    """
    return [name for name, node in self.nodes.items() if not node.depends_on]

# Add to DependencyGraph class
DependencyGraph.get_independent_nodes = get_independent_nodes

### 🧪 Test Queries

In [ ]:
# What does 'report' need?
deps = graph.get_dependencies('report')
print(f"'report' depends on: {sorted(deps)}")

# What needs 'extract'?
dependents = graph.get_dependents('extract')
print(f"'extract' is needed by: {sorted(dependents)}")

# Entry points
independent = graph.get_independent_nodes()
print(f"Independent nodes: {independent}")

## Part 7: Visualization

In [ ]:
def visualize(self) -> str:
    """Generate a text visualization of the graph.
    
    Returns:
        String representation of the graph
    """
    lines = ["Dependency Graph:", ""]
    
    # Show execution layers
    layers = self.get_execution_layers()
    for i, layer in enumerate(layers):
        lines.append(f"Layer {i + 1}:")
        for node_name in sorted(layer):
            node = self.nodes[node_name]
            deps = (
                f" (depends on: {', '.join(sorted(node.depends_on))})"
                if node.depends_on
                else ""
            )
            lines.append(f"  - {node_name}{deps}")
        lines.append("")
    
    return "\n".join(lines)

# Add to DependencyGraph class
DependencyGraph.visualize = visualize

In [ ]:
print(graph.visualize())

## Part 8: Complete Example - Complex Pipeline

In [ ]:
# Realistic data pipeline
pipeline_nodes = [
    # Sources
    NodeConfig('extract_customers', []),
    NodeConfig('extract_orders', []),
    NodeConfig('extract_products', []),
    
    # Cleaning
    NodeConfig('clean_customers', ['extract_customers']),
    NodeConfig('clean_orders', ['extract_orders']),
    NodeConfig('clean_products', ['extract_products']),
    
    # Validation
    NodeConfig('validate_customers', ['clean_customers']),
    NodeConfig('validate_orders', ['clean_orders']),
    
    # Enrichment
    NodeConfig('enrich_orders', ['validate_orders', 'clean_products']),
    
    # Aggregation
    NodeConfig('customer_metrics', ['validate_customers', 'enrich_orders']),
    NodeConfig('product_metrics', ['enrich_orders']),
    
    # Reporting
    NodeConfig('executive_report', ['customer_metrics', 'product_metrics'])
]

pipeline = DependencyGraph(pipeline_nodes)
print(pipeline.visualize())

print("\n" + "="*50)
print("Analysis:")
print("="*50)
print(f"Total nodes: {len(pipeline.nodes)}")
print(f"Total layers: {len(pipeline.get_execution_layers())}")
print(f"Max parallelism: {max(len(layer) for layer in pipeline.get_execution_layers())} nodes")
print(f"\n'executive_report' requires: {sorted(pipeline.get_dependencies('executive_report'))}")

## 🎓 Key Takeaways

1. **Two adjacency lists** enable efficient forward/backward traversal
2. **Validation first** catches errors before execution
3. **Topological sort** guarantees valid execution order
4. **Execution layers** maximize parallelism
5. **BFS queries** find transitive dependencies/dependents

## 🚀 Next Steps

1. Complete `exercises.ipynb` - build it from scratch
2. Read `advanced_graph_algorithms.md` for optimizations
3. Move to `06_error_handling` to handle execution failures